In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/}"
)  # just to enable `dataset`

import tensorflow as tf
import optuna
import dataset
import numpy as np
import keras
import joblib
from datetime import datetime
from keras import layers

2024-02-05 19:39:37.460325: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-05 19:39:37.460358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-05 19:39:37.461243: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEFAULT_BATCH_SIZE = 64
EPOCHS = 100
DEFAULT_INPUT_SIZE = 128

train_images_rgx = "../../dataset/train_set_4x/*/*.jpeg"
test_images_rgx = "../../dataset/test_set_4x/*/*.jpeg"

train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, DEFAULT_BATCH_SIZE, DEFAULT_INPUT_SIZE
)

In [3]:


def create_model_checkpointer(save_path):
    return keras.callbacks.ModelCheckpoint(
        save_path + "weights/",
        "val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )


def crate_early_stopper():
    return keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0.05,
        patience=20,
        verbose=0,
        mode="max",
        start_from_epoch=50,
    )

In [4]:
EXP_ID = "manual_test_4x"
batch_size = 64
input_size = 128

# load data
train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, batch_size, input_size
)

x = x_in = layers.Input(shape=(input_size, input_size, 3))

x = keras.layers.Conv2D(4, 4, 4)(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.Conv2D(16, 3, 1)(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(num_classes)(x)

# Build model and optimizer.
model = keras.Model(x_in, x)
assert isinstance(model, keras.Model), "model is not a model"

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 4)         196       
                                                                 
 activation (Activation)     (None, 32, 32, 4)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 16)        592       
                                                                 
 activation_1 (Activation)   (None, 30, 30, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 16)        0         
 D)                                                              
                                                             

In [5]:

optimizer = keras.optimizers.Adam()

# compile the model
model.compile(
    optimizer,
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)



In [6]:
# crate model checkpointer
saved_best_model_path = f"saved_models/{EXP_ID}/best_model_t/"
model.save(saved_best_model_path, True, "tf")  # save all model

model_chkp = create_model_checkpointer(saved_best_model_path)
early_stopper = crate_early_stopper()

# Training and validating cycle.
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS,
    callbacks=[model_chkp, early_stopper],
    verbose="auto",
)

# load best model
model.load_weights(saved_best_model_path + "weights/")


INFO:tensorflow:Assets written to: saved_models/manual_test_4x/best_model_t/assets


INFO:tensorflow:Assets written to: saved_models/manual_test_4x/best_model_t/assets


Epoch 1/100


I0000 00:00:1707172781.682236  555633 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


19/19 [==============================] - 3s 17ms/step - loss: 1.5321 - accuracy: 0.2368 - val_loss: 1.4958 - val_accuracy: 0.2692
Epoch 2/100
19/19 [==============================] - 0s 6ms/step - loss: 1.4175 - accuracy: 0.2500 - val_loss: 1.3932 - val_accuracy: 0.2923
Epoch 3/100
19/19 [==============================] - 0s 6ms/step - loss: 1.3312 - accuracy: 0.3602 - val_loss: 1.2586 - val_accuracy: 0.4692
Epoch 4/100
19/19 [==============================] - 0s 6ms/step - loss: 1.1407 - accuracy: 0.5995 - val_loss: 1.0033 - val_accuracy: 0.6769
Epoch 5/100
19/19 [==============================] - 0s 6ms/step - loss: 0.9246 - accuracy: 0.6661 - val_loss: 0.8704 - val_accuracy: 0.6846
Epoch 6/100
19/19 [==============================] - 0s 6ms/step - loss: 0.7787 - accuracy: 0.7048 - val_loss: 0.6950 - val_accuracy: 0.7846
Epoch 7/100
19/19 [==============================] - 0s 5ms/step - loss: 0.6726 - accuracy: 0.7459 - val_loss: 0.6567 - val_accuracy: 0.7385
Epoch 8/100
19/19 [=====

In [9]:

# Evaluate the model accuracy on the validation set.
score = model.evaluate(test_dataset, verbose=1)

3/3 [==============================] - 0s 3ms/step - loss: 0.4215 - accuracy: 0.8538


In [10]:
model.save(f"saved_models/{EXP_ID}/model.h5")

/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
